In [2]:
import glob 
import csv
import pandas as pd
import numpy as np
from scipy import stats 

import plotly.graph_objects as go
import plotly.express as px

pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 50)

What is the Tennessee TCAP (TNReady)? The Tennessee Comprehensive Assessment Program (TCAP) is the long running assessment test program in Tennessee. This program includes the TNReady (Tennessee Ready) which tests overall student progress in English Language Arts and Mathematics as well as Social Studies and Science.

MSAA: The Multi-State Alternate Assessment (MSAA) are ELA and mathematics assessments
designed for students with significant cognitive disabilities in grades 3–8 and grade 11. 

EOC: Each subject-area test is divided into multiple subparts and will be administered during one testing window at the end of the course.


Definitions
• Enrolled counts include the number of tested and non-tested records. Enrolled counts include all
students enrolled to take the test, regardless of test completion.
• Tested counts are the number of students that participated in the test.
• Valid test counts are the number of students that completed enough of the test to receive a valid
score.

Frequently Asked Questions

Why would each test/grade have different enrolled numbers?

There are many cases where student enrollment counts in a grade might differ across subjects. For
example, alternative tests are not included in these data, and some students may use general education
tests in some subjects and alternative tests in others.
What is the difference between "**" and "*"?


• The * is used if the number of valid tests is below 10.

• The ** suppression hides percentages (% Below, % Approaching, etc.) below 1% or above 99%.

In [3]:
merged_csvs = glob.glob('../raw_data_files_clean/merged_*')
merged_csvs.sort()

merged_csvs

['../raw_data_files_clean/merged_act_school.csv',
 '../raw_data_files_clean/merged_assessment.csv',
 '../raw_data_files_clean/merged_census_school.csv',
 '../raw_data_files_clean/merged_chronic_absenteeism_school.csv',
 '../raw_data_files_clean/merged_profile.csv',
 '../raw_data_files_clean/merged_tvaas_composite_school.csv',
 '../raw_data_files_clean/merged_tvaas_composite_subject_school.csv']

In [4]:
def pull_files_and_create_df(raw_file_names): 
    df_list=[]
    for i in raw_file_names:
        name = i.split('/')[2].split('.')[0]
        globals()[f'df_{name}'] = pd.read_csv(i, low_memory=False)
        d = df_list.append(f'df_{name}')
    df_list.sort(reverse=True)
    return df_list

In [5]:
pull_files_and_create_df(merged_csvs)

['df_merged_tvaas_composite_subject_school',
 'df_merged_tvaas_composite_school',
 'df_merged_profile',
 'df_merged_chronic_absenteeism_school',
 'df_merged_census_school',
 'df_merged_assessment',
 'df_merged_act_school']

In [6]:
df_merged_profile.loc[(df_merged_profile['school_number']==8030)]

,african_american_pct,asian_pct,district_number,district_name,economically_disadvantaged_pct,female_pct,hawaiian_pacisld_pct,hispanic_pct,limited_english_proficient_pct,male_pct,native_american_pct,school_number,school_name,students_with_disabilities_pct,total,white_pct,file_source,file_year,updated_district_number,school_type,status
16492,99.5798319327731,NaN,791,Memphis - old,86.5546218487395,52.5210084033614,NaN,NaN,NaN,47.4789915966387,NaN,8030,Star Academy,5.04201680672269,238.0,NaN,df_profile_school_2013,2013,791.0,Public Charter,I
18268,-,-,791,Memphis,77.9,-,NaN,-,NaN,-,-,8030,Star Academy,NaN,-,-,df_profile_school_2012,2012,791.0,Public Charter,I
20029,99.6,-,791,Memphis,-,55.5,NaN,0.4,NaN,44.5,-,8030,Star Academy,NaN,229,-,df_profile_school_2011,2011,791.0,Public Charter,I
21790,100,-,791,Memphis,-,57.3,NaN,-,NaN,42.7,-,8030,Star Academy,NaN,239.0,-,df_profile_school_2010,2010,791.0,Public Charter,I


In [7]:
profile_2021 = df_merged_profile.loc[(df_merged_profile['file_year']==2021) & (df_merged_profile['updated_district_number']==190)].copy()
profile_2021 = profile_2021[['school_number','african_american_pct','hispanic_pct','native_american_pct','economically_disadvantaged_pct','limited_english_proficient_pct','students_with_disabilities_pct']]
profile_2021 = profile_2021.replace(['','Less than 5%','*','Less than 1%','Greater than 95%','> 95%'],
                                              [0,0,0,0,95,95])
profile_2021

,school_number,african_american_pct,hispanic_pct,native_american_pct,economically_disadvantaged_pct,limited_english_proficient_pct,students_with_disabilities_pct
246,1,33.9,24.8,0,37.2,38.9,11.3
247,3,21.3,0,0,14.9,0,0
248,5,83.8,9,0,72.2,14.3,13.5
249,10,39.7,48.9,0,53.4,38.2,8.1
250,15,28.4,14.4,0,25.6,8.4,12.7
...,...,...,...,...,...,...,...
400,8105,36.5,47,0,30.4,43.5,12.2
1783,8005,81,13.2,0,77,19.9,17.2
1794,8090,43.5,42.9,0,50.9,34.4,14.6
1804,8015,37.9,24.6,0,28.4,28.2,10.2


In [8]:
df_merged_tvaas_composite_subject_school['school_type'] = df_merged_tvaas_composite_subject_school['school_type'].replace('Public Virtual School','Public')
df_merged_tvaas_composite_school['school_type'] = df_merged_tvaas_composite_school['school_type'].replace('Public Virtual School','Public')

In [9]:
df_merged_tvaas_composite_subject_school.loc[(df_merged_tvaas_composite_subject_school['school_number']==8030) & 
                                             (df_merged_tvaas_composite_subject_school['year']==2022)
                                            ]

,district,district_number,grade,growth_measure,index,level,number_of_students,school,school_number,standard_error,subject,test,year,file_source,file_year,updated_district_number,school_type,status
18011,Tennessee Public Charter School Commission,987,4.0,3.9,1.75,Level 4,40,Nashville Collegiate Prep,8030,2.2,English Language Arts,Grades 3-8,2022,df_tvaas_composite_subject_school_2022,2022,190,Public Charter,A
18012,Tennessee Public Charter School Commission,987,5.0,-4.8,-1.47,Level 2,22,Nashville Collegiate Prep,8030,3.2,English Language Arts,Grades 3-8,2022,df_tvaas_composite_subject_school_2022,2022,190,Public Charter,A
18013,Tennessee Public Charter School Commission,987,4.0,1.8,0.87,Level 3,41,Nashville Collegiate Prep,8030,2.0,Math,Grades 3-8,2022,df_tvaas_composite_subject_school_2022,2022,190,Public Charter,A
18014,Tennessee Public Charter School Commission,987,5.0,-3.6,-1.28,Level 2,21,Nashville Collegiate Prep,8030,2.8,Math,Grades 3-8,2022,df_tvaas_composite_subject_school_2022,2022,190,Public Charter,A
18015,Tennessee Public Charter School Commission,987,5.0,-6.1,-1.6,Level 2,14,Nashville Collegiate Prep,8030,3.8,Science,Grades 3-8,2022,df_tvaas_composite_subject_school_2022,2022,190,Public Charter,A


In [10]:
df_merged_tvaas_composite_subject_school['file_year'].unique()

array([2022, 2021, 2019, 2018, 2017])

In [24]:
mnps_tvaas_composite = df_merged_tvaas_composite_school.loc[(df_merged_tvaas_composite_school['updated_district_number'] == 190) & (df_merged_tvaas_composite_school['file_year'] == 2022)]

In [25]:
mnps_tvaas_composite

,district_name,district_number,literacy_and_numeracy_composite,literacy_composite,numeracy_composite,overall_composite,school_name,school_number,file_source,file_year,updated_district_number,school_type,status
131,Davidson County,190,3.0,3.0,2.0,3,A. Z. Kelley Elementary,1,df_tvaas_composite_school_2022,2022,190,Public,A
132,Davidson County,190,5.0,5.0,5.0,5,Amqui Elementary,10,df_tvaas_composite_school_2022,2022,190,Public,A
133,Davidson County,190,3.0,5.0,1.0,3,Chadwell Elementary,105,df_tvaas_composite_school_2022,2022,190,Public,A
134,Davidson County,190,5.0,5.0,5.0,5,Charlotte Park Elementary,110,df_tvaas_composite_school_2022,2022,190,Public,A
135,Davidson County,190,3.0,3.0,4.0,3,Eagle View Elementary School,115,df_tvaas_composite_school_2022,2022,190,Public,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...
279,Davidson County,190,5.0,5.0,5.0,5,Cane Ridge Elementary,82,df_tvaas_composite_school_2022,2022,190,Public,A
1721,Achievement School District,985,5.0,4.0,5.0,5,Brick Church: A LEAD Public School,8005,df_tvaas_composite_school_2022,2022,190,Public Charter,A
1731,Achievement School District,985,5.0,3.0,5.0,5,Neely's Bend: A LEAD Public School,8090,df_tvaas_composite_school_2022,2022,190,Public Charter,A
1741,Tennessee Public Charter School Commission,987,5.0,5.0,5.0,5,KIPP Antioch College Prep Middle,8020,df_tvaas_composite_school_2022,2022,190,Public Charter,A


In [32]:
mnps_tvaas_overall_composite_grouped = (
    mnps_tvaas_composite[['school_number','school_type','overall_composite']]
    .groupby(by=['school_type','overall_composite'])
    .count()
    .reset_index()
    .rename(columns={'school_number':'total_count'})
)

mnps_tvaas_overall_composite_grouped

,school_type,overall_composite,total_count
0,Public,1,22
1,Public,2,14
2,Public,3,35
3,Public,4,9
4,Public,5,42
5,Public Charter,1,3
6,Public Charter,2,2
7,Public Charter,3,2
8,Public Charter,5,24


In [27]:
mnps_tvaas_ela_overall_composite_grouped = (
    mnps_tvaas_composite[['school_number','school_type','literacy_composite']]
    .groupby(by=['school_type','literacy_composite'])
    .count()
    .reset_index()
    .rename(columns={'school_type':'school_type_ela','school_number':'ela_count'})
)

mnps_tvaas_ela_overall_composite_grouped

,school_type_ela,literacy_composite,ela_count
0,Public,1.0,8
1,Public,2.0,12
2,Public,3.0,45
3,Public,4.0,21
4,Public,5.0,35
5,Public Charter,2.0,2
6,Public Charter,3.0,6
7,Public Charter,4.0,2
8,Public Charter,5.0,21


In [28]:
mnps_tvaas_math_overall_composite_grouped = (
    mnps_tvaas_composite[['school_number','school_type','numeracy_composite']]
    .groupby(by=['school_type','numeracy_composite'])
    .count()
    .reset_index()
    .rename(columns={'school_type':'school_type_math','school_number':'math_count'})
)

mnps_tvaas_math_overall_composite_grouped

,school_type_math,numeracy_composite,math_count
0,Public,1.0,21
1,Public,2.0,8
2,Public,3.0,20
3,Public,4.0,20
4,Public,5.0,53
5,Public Charter,1.0,3
6,Public Charter,2.0,2
7,Public Charter,3.0,3
8,Public Charter,5.0,23


In [29]:
grouped_composite = pd.concat([mnps_tvaas_overall_composite_grouped,mnps_tvaas_ela_overall_composite_grouped,mnps_tvaas_math_overall_composite_grouped],axis=1)
grouped_composite = grouped_composite[['school_type','overall_composite','total_count','ela_count','math_count']]

In [31]:
grouped_composite

,school_type,overall_composite,total_count,ela_count,math_count
0,Public,1,22,8,21
1,Public,2,14,12,8
2,Public,3,35,45,20
3,Public,4,9,21,20
4,Public,5,42,35,53
5,Public Charter,1,3,2,3
6,Public Charter,2,2,6,2
7,Public Charter,3,2,2,3
8,Public Charter,5,24,21,23


In [30]:
df_merged_tvaas_composite_school['school_type'].unique()

array(['Public', 'Public Charter', 'State Special School', 'Grade school'],
      dtype=object)

In [18]:
mnps_tvaas_math_ela = df_merged_tvaas_composite_subject_school.loc[(df_merged_tvaas_composite_subject_school['updated_district_number'] == 190) 
                                                                   & ((df_merged_tvaas_composite_subject_school['subject']=='English Language Arts') | (df_merged_tvaas_composite_subject_school['subject']=='Math')) 
                                                                  & (df_merged_tvaas_composite_subject_school['file_year']==2022)]
mnps_tvaas_math_ela[mnps_tvaas_math_ela['school_number']==8030]


## Growth Index - TVAAS is calculated by comparing a student's performance on a state assessment to the performance of peers who have a similar testing history. The difference between how a student would be expected to perform and his or her actual performance results in an annual measure of growth.

,district,district_number,grade,growth_measure,index,level,number_of_students,school,school_number,standard_error,subject,test,year,file_source,file_year,updated_district_number,school_type,status
18011,Tennessee Public Charter School Commission,987,4.0,3.9,1.75,Level 4,40,Nashville Collegiate Prep,8030,2.2,English Language Arts,Grades 3-8,2022,df_tvaas_composite_subject_school_2022,2022,190,Public Charter,A
18012,Tennessee Public Charter School Commission,987,5.0,-4.8,-1.47,Level 2,22,Nashville Collegiate Prep,8030,3.2,English Language Arts,Grades 3-8,2022,df_tvaas_composite_subject_school_2022,2022,190,Public Charter,A
18013,Tennessee Public Charter School Commission,987,4.0,1.8,0.87,Level 3,41,Nashville Collegiate Prep,8030,2.0,Math,Grades 3-8,2022,df_tvaas_composite_subject_school_2022,2022,190,Public Charter,A
18014,Tennessee Public Charter School Commission,987,5.0,-3.6,-1.28,Level 2,21,Nashville Collegiate Prep,8030,2.8,Math,Grades 3-8,2022,df_tvaas_composite_subject_school_2022,2022,190,Public Charter,A


In [19]:
mnps_tvaas_math_ela = mnps_tvaas_math_ela.merge(profile_2021, on='school_number', how='left')

In [20]:
# stats.ttest_ind(mnps_all_grades.loc[(df_all_grades['school_type']=='Public') & (mnps_all_grades['subject_group']=='Math') & (mnps_all_grades['file_year']==2022)]['pct_met_exceeded_adjusted'],
#                 mnps_all_grades.loc[(df_all_grades['school_type']=='Public Charter') & (mnps_all_grades['subject_group']=='Math') & (mnps_all_grades['file_year']==2022)]['pct_met_exceeded_adjusted'], trim=.2)

In [33]:
with pd.ExcelWriter('../data_for_analysis/tvaas.xlsx') as writer:  
    mnps_tvaas_math_ela.to_excel(writer, sheet_name='math_ela_school_scores')
    grouped_composite.to_excel(writer, sheet_name='grouped_composite')
    mnps_tvaas_composite.to_excel(writer, sheet_name='school_level_tvaas')